The ipynb builds a model using the exising tsgm agent cgan

In [ ]:
import tsgm 
import tensorflow as tf
from tensorflow import keras
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)

memory_limit = 6000
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]
        )
        print(f"Memory limit set to {memory_limit} MB for GPU.")
    except RuntimeError as e:
        print(e)

import pandas as pd 
import numpy as np
from tensorflow.keras.layers import Reshape



In [ ]:
latent_dim = 100
seq_len = 100
feature_dim = 2

dataset = np.load("/home/junze/.jupyter/Data transfer and loading/dataset_1.csv.npy")
print("Dataset shape:", dataset.shape)
seq_len, feature_dim = dataset.shape[1], dataset.shape[2]
latent_dim = 100

# Create a custom encoder
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def create_custom_encoder(seq_len, feature_dim, latent_dim):
    inputs = tf.keras.Input(shape=(seq_len, feature_dim))
    x = tf.keras.layers.Conv1D(32, 3, activation="relu", padding="same")(inputs)
    x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    
    z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    
    encoder = tf.keras.Model(inputs=inputs, outputs=[z_mean, z_log_var, z])
    return encoder

# Create a custom decoder
def create_custom_decoder(seq_len, feature_dim, latent_dim):
    latent_inputs = tf.keras.Input(shape=(latent_dim,))
    x = tf.keras.layers.Dense(seq_len * 64, activation="relu")(latent_inputs)
    x = tf.keras.layers.Reshape((seq_len, 64))(x)
    x = tf.keras.layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv1D(16, 3, activation="relu", padding="same")(x)
    outputs = tf.keras.layers.Conv1D(feature_dim, 1, activation="sigmoid")(x)
    
    decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs)
    return decoder

encoder = create_custom_encoder(seq_len, feature_dim, latent_dim)
decoder = create_custom_decoder(seq_len, feature_dim, latent_dim)

# Create the BetaVAE
cvae = tsgm.models.cvae.BetaVAE(encoder, decoder)

print("Encoder summary:")
encoder.summary()

print("\nDecoder summary:")
decoder.summary()

# Compile and fit the model
cvae.compile(optimizer=tf.keras.optimizers.Adam())
cvae.fit(dataset, epochs=1000, batch_size=64)

In [ ]:
cvae.compile()
cvae.fit(dataset, epochs=1000, batch_size=64)